In [2]:
!pip install lxml

  Using cached lxml-6.0.2-cp310-cp310-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.6 kB)
Using cached lxml-6.0.2-cp310-cp310-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (5.3 MB)


In [10]:
import requests
import pandas as pd
import time

BASE_URL = "https://kind.krx.co.kr/corpgeneral/corpList.do"

def fetch_krx_corp_list(page_size=100, sleep_sec=0.2, max_pages=10):
    """
    KIND 상장법인 목록 전체 수집
    - page_size: currentPageSize (페이지당 건수)
    - sleep_sec: 요청 간 대기 시간(서버 부담 줄이기용)
    - max_pages: 무한루프 방지용 상한
    """
    all_dfs = []

    headers = {
        # KRX가 User-Agent 없으면 차단하는 경우가 있어서 같이 보냄
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    for page in range(1, max_pages + 1):
        params = {
            "method": "searchCorpList",
            "pageIndex": page,
            "currentPageSize": page_size,
            "comAbbrv": "",
            "beginIndex": "",
            "orderMode": 3,
            "orderStat": "D",
            "isurCd": "",
            "repIsuSrtCd": "",
            "searchCodeType": "",
            "marketType": "",        # 코스피/코스닥/코넥스 구분값 (전체면 비움)
            "searchType": 13,        # 13: 상장법인
            "industry": "",
            "fiscalYearEnd": "all",
            "comAbbrvTmp": "",
            "location": "all"
        }

        print(f"요청 중... pageIndex={page}")

        resp = requests.get(BASE_URL, params=params, headers=headers)
        # KIND는 EUC-KR 인코딩인 경우가 많음
        resp.encoding = "utf-8"

        # HTML 안의 table을 판다스로 바로 파싱
        try:
            tables = pd.read_html(resp.text)
        except ValueError:
            print("더 이상 테이블이 없습니다. 종료합니다.")
            break

        if not tables:
            print("테이블이 비어 있습니다. 종료합니다.")
            break

        df = tables[0]

        # 데이터 없는 빈 페이지면 종료
        if df.empty:
            print("빈 페이지를 만났습니다. 종료합니다.")
            break

        # 첫 페이지가 아니면서, 이전 페이지와 완전히 동일하면 종료(안전장치)
        if all_dfs and df.equals(all_dfs[-1]):
            print("이전 페이지와 동일한 데이터입니다. 종료합니다.")
            break

        all_dfs.append(df)

        # 마지막 페이지 추정: 행 개수가 page_size보다 작으면 끝까지 온 것
        if len(df) < page_size:
            print("마지막 페이지로 판단되어 종료합니다.")
            break

        time.sleep(sleep_sec)

    if not all_dfs:
        return pd.DataFrame()

    result = pd.concat(all_dfs, ignore_index=True)
    return result


if __name__ == "__main__":
    # 페이지당 15개(질문에서 캡처한 값) 기준으로 전체 수집
    corp_df = fetch_krx_corp_list(page_size=15)

    print("총 행 수:", len(corp_df))
    print(corp_df.head())

    # CSV로 저장 (필요하면 경로만 바꿔서)
    corp_df.to_csv("krx_corp_list.csv", index=False, encoding="utf-8-sig")


요청 중... pageIndex=1


/tmp/ipykernel_3294/85557543.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


요청 중... pageIndex=2


/tmp/ipykernel_3294/85557543.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


요청 중... pageIndex=3


...

요청 중... pageIndex=9


/tmp/ipykernel_3294/85557543.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


요청 중... pageIndex=10


/tmp/ipykernel_3294/85557543.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


총 행 수: 150
   Unnamed: 0       Unnamed: 1  Unnamed: 2  Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0   엔에이치스팩32호       금융 지원 서비스업          합병  2025-12-05        12월        김수한   
1      에임드바이오  자연과학 및 공학 연구개발업  ADC 기반 항암제  2025-12-04        12월        허남구   
2  미래에셋비전스팩9호       금융 지원 서비스업      기업인수합병  2025-12-01        12월        황리건   
3     삼성스팩12호           기타 금융업      기업인수합병  2025-11-28        12월        김영제   
4  미래에셋비전스팩8호       금융 지원 서비스업      기업인수합병  2025-11-27        12월        김병철   

  Unnamed: 6 Unnamed: 7  
0        NaN      서울특별시  
1    홈페이지 보기      서울특별시  
2        NaN      서울특별시  
3        NaN      서울특별시  
4        NaN      서울특별시  


In [11]:
corp_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,엔에이치스팩32호,금융 지원 서비스업,합병,2025-12-05,12월,김수한,NaN,서울특별시
1,에임드바이오,자연과학 및 공학 연구개발업,ADC 기반 항암제,2025-12-04,12월,허남구,홈페이지 보기,서울특별시
2,미래에셋비전스팩9호,금융 지원 서비스업,기업인수합병,2025-12-01,12월,황리건,NaN,서울특별시
3,삼성스팩12호,기타 금융업,기업인수합병,2025-11-28,12월,김영제,NaN,서울특별시
4,미래에셋비전스팩8호,금융 지원 서비스업,기업인수합병,2025-11-27,12월,김병철,NaN,서울특별시
...,...,...,...,...,...,...,...,...
145,웨이비스,반도체 제조업,"GaN RF 칩, 패키지 트랜지스터, 모듈 등",2024-10-25,12월,한민석,홈페이지 보기,경기도
146,씨메스,특수 목적용 기계 제조업,AI 기반 로봇 솔루션 및 3D 비전 솔루션,2024-10-24,12월,이성호,홈페이지 보기,서울특별시
147,한켐,기초 화학물질 제조업,OLED 발광층 소재 등,2024-10-22,12월,이상조,홈페이지 보기,대전광역시
148,루미르,"항공기,우주선 및 부품 제조업","인공위성 시스템 및 전장품, 위성 영상 및 정보",2024-10-21,12월,남명용,홈페이지 보기,경기도


In [12]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup

BASE_URL = "https://kind.krx.co.kr/corpgeneral/corpList.do"

def fetch_krx_corp_list(page_size=100, sleep_sec=0.2, max_pages=10):
    """
    KIND 상장법인 목록 전체 수집
    - 분류(코스피/코스닥 등): <img alt="코스닥">
    - 종목코드: onclick="companysummary_open('0009K');"
    - 홈페이지: <a class="homepage" href="...">
    """

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    rows_data = []

    for page in range(1, max_pages + 1):
        params = {
            "method": "searchCorpList",
            "pageIndex": page,
            "currentPageSize": page_size,
            "comAbbrv": "",
            "beginIndex": "",
            "orderMode": 3,
            "orderStat": "D",
            "isurCd": "",
            "repIsuSrtCd": "",
            "searchCodeType": "",
            "marketType": "",
            "searchType": 13,
            "industry": "",
            "fiscalYearEnd": "all",
            "comAbbrvTmp": "",
            "location": "all",
        }

        print(f"요청 중... pageIndex={page}")
        resp = requests.get(BASE_URL, params=params, headers=headers)
        resp.encoding = "utf-8"

        soup = BeautifulSoup(resp.text, "html.parser")

        # 회사 행들: id="companysum" a가 포함된 tr만 사용
        tr_list = soup.find_all("tr")
        page_count = 0

        for tr in tr_list:
            a_company = tr.find("a", id="companysum")
            if not a_company:
                continue  # 헤더/기타 행은 스킵

            tds = tr.find_all("td")
            if len(tds) < 8:
                # 예상되는 열 개수보다 적으면 스킵
                continue

            # 1) first td (회사명, 분류, 종목코드)
            first_td = tr.find("td", class_="first")

            # (1) 회사명
            corp_name = a_company.get_text(strip=True)

            # (2) 분류 (코스피/코스닥 등) - <img alt="코스닥">
            img = first_td.find("img")
            category = img.get("alt", "").strip() if img else ""

            # (3) 종목코드 - onclick="companysummary_open('0009K');"
            onclick = a_company.get("onclick", "")
            m = re.search(r"companysummary_open\('([^']+)'\)", onclick)
            corp_code = m.group(1) if m else ""

            # 2) 나머지 열 (업종, 주요제품, 상장일, 결산월, 대표자, 홈페이지, 지역)
            industry = tds[1].get_text(strip=True)
            product = tds[2].get_text(strip=True)
            list_date = tds[3].get_text(strip=True)       # 상장일
            fiscal_month = tds[4].get_text(strip=True)    # 결산월
            ceo = tds[5].get_text(strip=True)             # 대표자명
            region = tds[7].get_text(strip=True)          # 지역

            # 3) 홈페이지 URL: <a class="btn ico homepage" href="http://www.aimedbio.com" ...>
            homepage_a = tr.find("a", class_="homepage")
            homepage = homepage_a.get("href", "").strip() if homepage_a else ""

            rows_data.append({
                "회사명": corp_name,
                "분류": category,          # 코스피 / 코스닥 등
                "종목코드": corp_code,
                "업종": industry,
                "주요제품": product,
                "상장일": list_date,
                "결산월": fiscal_month,
                "대표자명": ceo,
                "홈페이지": homepage,
                "지역": region,
            })

            page_count += 1

        if page_count == 0:
            # 이 페이지에 더 이상 데이터가 없으면 종료
            print("이 페이지에서 유효한 데이터가 없습니다. 종료합니다.")
            break

        # page_size보다 적으면 마지막 페이지로 판단
        if page_count < page_size:
            print("마지막 페이지로 판단되어 종료합니다.")
            break

        time.sleep(sleep_sec)

    # DataFrame으로 변환
    df = pd.DataFrame(rows_data)
    return df


if __name__ == "__main__":
    corp_df = fetch_krx_corp_list(page_size=15)

    print("총 행 수:", len(corp_df))
    print(corp_df.head())

    # CSV로 저장
    display(corp_df)


요청 중... pageIndex=1
요청 중... pageIndex=2
요청 중... pageIndex=3
요청 중... pageIndex=4
요청 중... pageIndex=5
요청 중... pageIndex=6
요청 중... pageIndex=7
요청 중... pageIndex=8
요청 중... pageIndex=9
요청 중... pageIndex=10
총 행 수: 150
          회사명   분류   종목코드               업종        주요제품         상장일  결산월 대표자명  \
0   엔에이치스팩32호  코스닥  0054V       금융 지원 서비스업          합병  2025-12-05  12월  김수한   
1      에임드바이오  코스닥  0009K  자연과학 및 공학 연구개발업  ADC 기반 항암제  2025-12-04  12월  허남구   
2  미래에셋비전스팩9호  코스닥  0096D       금융 지원 서비스업      기업인수합병  2025-12-01  12월  황리건   
3     삼성스팩12호  코스닥  0096B           기타 금융업      기업인수합병  2025-11-28  12월  김영제   
4  미래에셋비전스팩8호  코스닥  0093G       금융 지원 서비스업      기업인수합병  2025-11-27  12월  김병철   

                      홈페이지     지역  
0                           서울특별시  
1  http://www.aimedbio.com  서울특별시  
2                           서울특별시  
3                           서울특별시  
4                           서울특별시  


,회사명,분류,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,엔에이치스팩32호,코스닥,0054V,금융 지원 서비스업,합병,2025-12-05,12월,김수한,,서울특별시
1,에임드바이오,코스닥,0009K,자연과학 및 공학 연구개발업,ADC 기반 항암제,2025-12-04,12월,허남구,http://www.aimedbio.com,서울특별시
2,미래에셋비전스팩9호,코스닥,0096D,금융 지원 서비스업,기업인수합병,2025-12-01,12월,황리건,,서울특별시
3,삼성스팩12호,코스닥,0096B,기타 금융업,기업인수합병,2025-11-28,12월,김영제,,서울특별시
4,미래에셋비전스팩8호,코스닥,0093G,금융 지원 서비스업,기업인수합병,2025-11-27,12월,김병철,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
145,웨이비스,코스닥,28993,반도체 제조업,"GaN RF 칩, 패키지 트랜지스터, 모듈 등",2024-10-25,12월,한민석,http://www.wavice.com/,경기도
146,씨메스,코스닥,47540,특수 목적용 기계 제조업,AI 기반 로봇 솔루션 및 3D 비전 솔루션,2024-10-24,12월,이성호,http://www.cmesrobotics.ai/,서울특별시
147,한켐,코스닥,45737,기초 화학물질 제조업,OLED 발광층 소재 등,2024-10-22,12월,이상조,http://www.hanchem.net,대전광역시
148,루미르,코스닥,47417,"항공기,우주선 및 부품 제조업","인공위성 시스템 및 전장품, 위성 영상 및 정보",2024-10-21,12월,남명용,http://www.lumir.space,경기도


In [ ]:
corp_df